In [23]:
import requests     
import json         
import pandas as pd 
import time
from fuzzywuzzy import fuzz

In [2]:
path = "./Data/withEOD.csv"

df = pd.read_csv(path)  
df

,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,country-Inv,...,symbol-Inv,asset_class-Inv,Name_197,industry-IB,longName-IB,marketName-IB,Symbol-IB,Country-IB,Exchange-IB,Currency-IB
0,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,united states,...,TWOU,equity,2U INC,"Consumer, Non-cyclical",2U INC,NMS,TWOU,NaN,NaN,NaN
1,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,united kingdom,...,88E,equity,88 ENERGY LTD,Energy,88 ENERGY LTD,88E,88E,United Kingdom,LSE,GBP
2,7063,Japan,JPY,TSE,JP3160590000,A Dot Co Ltd,Common Stock,2064.000,2022-08-03,japan,...,7063,equity,A DOT CO LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,Hong Kong,HKD,HK,KYG2953R1149,AAC Technologies Holdings Inc,Common Stock,14.580,2022-08-03,hong kong,...,2018,equity,AAC TECHNOLOGIES HOLDINGS IN,Industrial,AAC TECHNOLOGIES HOLDINGS IN,2018,2018,NaN,NaN,NaN
4,2686,Hong Kong,HKD,HK,KYG000371040,AAG Energy Holdings Ltd,Common Stock,1.410,2022-08-03,hong kong,...,2686,equity,AAG ENERGY HOLDINGS LTD,Energy,AAG ENERGY HOLDINGS LTD,2686,2686,Hong Kong,SEHK,HKD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,NaN,NaN,NaN,NaN,US98979J1097,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZOE'S KITCHEN INC,"Consumer, Cyclical",ZOE'S KITCHEN INC,ZOES,ZOES,NaN,NaN,NaN
17507,NaN,NaN,NaN,NaN,CA98977K1084,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZOLOTO RESOURCES LTD,Basic Materials,ZOLOTO RESOURCES LTD,ZR,ZR,NaN,NaN,NaN
17508,NaN,NaN,NaN,NaN,CA98977R1038,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZOOMMED INC,Technology,ZOOMMED INC,ZMD.H,ZMD.H,NaN,NaN,NaN
17509,NaN,NaN,NaN,NaN,GB00BMHTHT14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZPG PLC,Communications,ZPG PLC,ZPG,ZPG,NaN,NaN,NaN


In [13]:
len(ticker_list)

100

In [94]:
# key & urls
# openfigi_apikey = '71aaa940-3567-4040-bce5-9f43490a7703' # GAI
openfigi_apikey = '6531f095-b68a-44c2-9749-20b3b75f9ad8'
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [21]:
def map_jobs(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, 
                             headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()

In [95]:
def get100Figi(ticker_list):
    
    jobs = []
    for i in ticker_list:
        s = "{'idType': 'ID_ISIN', 'idValue': '" + str(i) + "'}"
        json_acceptable_string = s.replace("'", "\"")
        d = json.loads(json_acceptable_string)
        jobs.append(d)
    
    
    job_results = map_jobs(jobs)
    
    # Extract figi from scrape results
    figi_value = [] # store figi information
    figi_count = [] # keep track of the number of each symbol's corresponding figi
    j=0
    for d in job_results:
        if 'data' in d:
            figi_count.append(len(d['data']))
            for i in range(len(d['data'])):   
                figi_value.append(d['data'][i])
                d['data'][i]['ISIN'] = ticker_list[j]
        else:
#             d['data'][i]['ISIN'] = ticker_list[j]
            figi_count.append(0)
        j+=1
        
    df_figi = pd.DataFrame.from_dict(figi_value)
    
    # extract useful information and rename
    df_res = df_figi[['ISIN', 'name', 'ticker', 'figi', 'compositeFIGI', 'shareClassFIGI', 'exchCode', 'marketSector', 'securityDescription', 'securityType', 'securityType2']].rename(columns = {'name':'longName', 'ticker':'symbol'})
    
    return df_res


In [96]:
len(df)//100 + 1

176

In [97]:
requestTimes = len(df)//100 + 1
df_figi = pd.DataFrame()

for i in range(requestTimes):
    # get 100 tickers
    ticker_list = list(df['ISIN'].unique())[100*(i):100*(i+1)]
    # get figi
    df100 = get100Figi(ticker_list)
    df100 = df100.drop_duplicates(subset=['ISIN'])
    df_figi = pd.concat([df_figi, df100])
#     time.sleep()
    
    # print progress
    if i % (requestTimes//10) == 0:
        print("progress: %.2f" % (100*i/requestTimes) + "%")

df_figi

progress: 0.00%
progress: 9.66%
progress: 19.32%
progress: 28.98%
progress: 38.64%
progress: 48.30%
progress: 57.95%
progress: 67.61%
progress: 77.27%
progress: 86.93%
progress: 96.59%


,ISIN,longName,symbol,figi,compositeFIGI,shareClassFIGI,exchCode,marketSector,securityDescription,securityType,securityType2
0,US90214J1016,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
96,AU00000088E2,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
172,JP3160590000,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
179,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
251,KYG000371040,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...
3,US9895063082,ZIEGLER COS INC/THE,ZGCO,BBG000CG8F44,BBG000CG8F44,BBG001S6VMH2,US,Equity,ZGCO,Common Stock,Common Stock
6,CA98959W2031,ZINC ONE RESOURCES INC,Z/H,BBG000RG2ZZ6,BBG000RG2ZZ6,BBG001STKXP1,CN,Equity,Z/H,Common Stock,Common Stock
80,US9897018183,ZIONS BANCORP NA,ZION V6.95 09/15/28,BBG0058QYKL7,None,None,NASDAQ/NGM,Pfd,ZION 6.95 09/15/28,PUBLIC,Preferred Stock
81,CA98977R1038,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock


In [106]:
df_figi = df_figi.rename(columns = {'longName': 'longName-FIGI', 'symbol': 'symbol-FIGI', 'exchCode': 'exchCode-FIGI', 'marketSector': 'marketSector-FIGI', 'securityDescription': 'securityDescription-FIGI', 'securityType': 'securityType-FIGI', 'securityType2': 'securityType2-FIGI'})

df_figi 

,ISIN,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,US90214J1016,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
96,AU00000088E2,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
172,JP3160590000,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
179,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
251,KYG000371040,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...
3,US9895063082,ZIEGLER COS INC/THE,ZGCO,BBG000CG8F44,BBG000CG8F44,BBG001S6VMH2,US,Equity,ZGCO,Common Stock,Common Stock
6,CA98959W2031,ZINC ONE RESOURCES INC,Z/H,BBG000RG2ZZ6,BBG000RG2ZZ6,BBG001STKXP1,CN,Equity,Z/H,Common Stock,Common Stock
80,US9897018183,ZIONS BANCORP NA,ZION V6.95 09/15/28,BBG0058QYKL7,None,None,NASDAQ/NGM,Pfd,ZION 6.95 09/15/28,PUBLIC,Preferred Stock
81,CA98977R1038,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock


In [99]:
len(df['ISIN'].unique())

17511

### Merge to the dataset

In [107]:
df_withFIGI = pd.merge(df, df_figi, how='outer', on='ISIN')
df_withFIGI

,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,country-Inv,...,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,united states,...,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
1,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,united kingdom,...,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
2,7063,Japan,JPY,TSE,JP3160590000,A Dot Co Ltd,Common Stock,2064.000,2022-08-03,japan,...,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
3,2018,Hong Kong,HKD,HK,KYG2953R1149,AAC Technologies Holdings Inc,Common Stock,14.580,2022-08-03,hong kong,...,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
4,2686,Hong Kong,HKD,HK,KYG000371040,AAG Energy Holdings Ltd,Common Stock,1.410,2022-08-03,hong kong,...,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,NaN,NaN,NaN,NaN,US98979J1097,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17507,NaN,NaN,NaN,NaN,CA98977K1084,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17508,NaN,NaN,NaN,NaN,CA98977R1038,NaN,NaN,NaN,NaN,NaN,...,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock
17509,NaN,NaN,NaN,NaN,GB00BMHTHT14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_withFIGI.to_csv('./Data/withFIGI.csv', index=False)

## Check FIGI Name vs. 19.7k

In [111]:
df_Name = df_withFIGI[['ISIN', 'Name_197', 'longName-FIGI']].dropna(subset=['longName-FIGI'])

df_Name

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,ISIN,Name_197,longName-FIGI
0,US90214J1016,2U INC,2U INC
1,AU00000088E2,88 ENERGY LTD,88 ENERGY LTD
2,JP3160590000,A DOT CO LTD,BIRDMAN INC
3,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,AAC TECHNOLOGIES HOLDINGS IN
4,KYG000371040,AAG ENERGY HOLDINGS LTD,AAG ENERGY HOLDINGS LTD
...,...,...,...
17501,US9895063082,ZIEGLER COS INC/THE,ZIEGLER COS INC/THE
17502,CA98959W2031,ZINC ONE RESOURCES INC,ZINC ONE RESOURCES INC
17505,US9897018183,ZIONS BANCORPORATION,ZIONS BANCORP NA
17508,CA98977R1038,ZOOMMED INC,ZOOMMED INC


In [113]:
allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['longName-FIGI'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

progress: 0.00%
progress: 9.99%
progress: 19.99%
progress: 29.98%
progress: 39.98%
progress: 49.97%
progress: 59.97%
progress: 69.96%
progress: 79.95%
progress: 99.94%


In [114]:
df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match

,ISIN,Name_197,longName-FIGI,fuzz_socre1
0,US90214J1016,2U INC,2U INC,100
1,AU00000088E2,88 ENERGY LTD,88 ENERGY LTD,100
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
3,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,AAC TECHNOLOGIES HOLDINGS IN,100
4,KYG000371040,AAG ENERGY HOLDINGS LTD,AAG ENERGY HOLDINGS LTD,100
...,...,...,...,...
17501,US9895063082,ZIEGLER COS INC/THE,ZIEGLER COS INC/THE,100
17502,CA98959W2031,ZINC ONE RESOURCES INC,ZINC ONE RESOURCES INC,100
17505,US9897018183,ZIONS BANCORPORATION,ZIONS BANCORP NA,78
17508,CA98977R1038,ZOOMMED INC,ZOOMMED INC,100


In [115]:
df_match[df_match['fuzz_socre1']<80]

,ISIN,Name_197,longName-FIGI,fuzz_socre1
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
15,SE0006256558,ABSOLENT GROUP AB,ABSOLENT AIR CARE GROUP AB,79
38,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,32
53,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,58
57,US00486H1059,ADTRAN INC,ADTRAN HOLDINGS INC,69
...,...,...,...,...
16791,KYG735374020,SCOTTISH RE GROUP LTD PFD B 7.25%,SCOTTISH RE GROUP LTD,79
17055,US7847882005,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",STRATS-FON-04-2,36
17103,MHY8564M1057,TEEKAY LNG PARTNERS LP,SEAPEAK LLC,30
17110,US8794338520,TELEPHONE AND DATA SYSTEMS INC 6.625% PFD,TELEPHONE & DATA SYSTEMS,71


In [116]:
df_match[df_match['fuzz_socre1']<60]

,ISIN,Name_197,longName-FIGI,fuzz_socre1
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
38,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,32
53,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,58
78,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,25
121,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,57
...,...,...,...,...
16577,US74838C1062,QUIKSILVER INC,BOARDRIDERS INC,21
16581,US7472623013,QVC INC 6 3/8 09/13/67 PFD,QVC INC,42
16690,SE0010520155,ROLLING OPTICS HOLDING AB,ICTA AB,31
17055,US7847882005,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",STRATS-FON-04-2,36


## Check Company Name from different sources

In [5]:
path = "./Data/withFIGI.csv"

df_withFIGI = pd.read_csv(path)  
df_withFIGI

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,5,6,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,country-Inv,...,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,united states,...,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
1,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,united kingdom,...,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
2,7063,Japan,JPY,TSE,JP3160590000,A Dot Co Ltd,Common Stock,2064.000,2022-08-03,japan,...,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
3,2018,Hong Kong,HKD,HK,KYG2953R1149,AAC Technologies Holdings Inc,Common Stock,14.580,2022-08-03,hong kong,...,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
4,2686,Hong Kong,HKD,HK,KYG000371040,AAG Energy Holdings Ltd,Common Stock,1.410,2022-08-03,hong kong,...,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,NaN,NaN,NaN,NaN,US98979J1097,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17507,NaN,NaN,NaN,NaN,CA98977K1084,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17508,NaN,NaN,NaN,NaN,CA98977R1038,NaN,NaN,NaN,NaN,NaN,...,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock
17509,NaN,NaN,NaN,NaN,GB00BMHTHT14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### EOD & Investing.com

In [10]:

df_Name = df_withFIGI[['ISIN', 'full_name-Inv', 'Name-EOD']].dropna(subset=['full_name-Inv'])
df_Name = df_Name[['ISIN', 'full_name-Inv', 'Name-EOD']].dropna(subset=['Name-EOD'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['full_name-Inv'],row['Name-EOD'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

5266
progress: 0.00%
progress: 9.99%
progress: 19.98%
progress: 29.97%
progress: 39.95%
progress: 49.94%
progress: 59.93%
progress: 69.92%
progress: 79.91%
progress: 89.90%
progress: 99.89%


,ISIN,full_name-Inv,Name-EOD,fuzz_socre1
38,JP3108120001,Acrodea Inc,Why How Do Co Inc,50
53,LU1250154413,ADO Properties SA,Adler Group SA,58
78,US0077371096,AeroCentury Corp,Mega Matrix Corp,25
165,US0185811082,Alliance Data Systems Corp,Bread Financial Holdings Inc,26
223,US0259321042,American Financial Group Inc,Aquatic Foods Group plc,51
...,...,...,...,...
4544,KYG857AT1018,Sun International Group Ltd,Imperium Financial Group Ltd,51
4573,JP3397150008,Sushiro Global Holdings Ltd,Food & Life Companies Ltd,24
4757,JP3588600001,Tokyo Broadcasting System Inc,TBS Holdings Inc,49
5056,PLGPW0000017,Warsaw Stock Exchange SA,Gielda Papierow Wartosciowych w Warszawie SA,41


### Investing.com & IB

In [11]:

df_Name = df_withFIGI[['ISIN', 'full_name-Inv', 'longName-IB']].dropna(subset=['full_name-Inv'])
df_Name = df_Name[['ISIN', 'full_name-Inv', 'longName-IB']].dropna(subset=['longName-IB'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['full_name-Inv'],row['longName-IB'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

5563
progress: 0.00%
progress: 9.99%
progress: 19.99%
progress: 29.98%
progress: 39.98%
progress: 49.97%
progress: 59.97%
progress: 69.96%
progress: 89.95%
progress: 239.87%
progress: 249.87%


,ISIN,full_name-Inv,longName-IB,fuzz_socre1
246,IL0010806136,Analyst IMS Investment Management Services Ltd,ANALYST I.M.S.,44
562,RU000A0HL5M1,Beluga Group PAO,BELUGA,55
2779,US5006432000,Korn/Ferry International,KORN FERRY,59
2965,IL0004730177,Amos Luzon Development and Energy Group Ltd,LUZON GROUP,41
3794,PLPKO0000016,Powszechna Kasa Oszczednosci Bank Polski SA,PKO BANK POLSKI SA,56
4167,FR0006239109,Soc Centrale Bois Scieries Manche SA,SCBSM,15


### FIGI & IB

In [12]:
df_Name = df_withFIGI[['ISIN', 'longName-FIGI', 'longName-IB']].dropna(subset=['longName-FIGI'])
df_Name = df_Name[['ISIN', 'longName-FIGI', 'longName-IB']].dropna(subset=['longName-IB'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['longName-FIGI'],row['longName-IB'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

14946
progress: 0.00%
progress: 10.00%
progress: 19.99%
progress: 29.99%
progress: 39.98%
progress: 49.98%
progress: 59.98%
progress: 69.97%
progress: 79.97%
progress: 89.96%


,ISIN,longName-FIGI,longName-IB,fuzz_socre1
196,CA02210D1033,TENAZ ENERGY CORP,ALTURA ENERGY INC,59
370,US0441861046,ASHLAND INC,ASHLAND GLOBAL HOLDINGS INC,58
533,IL0007590198,GAV-YAM LANDS CORP LTD,BAYSIDE LAND CORPORATION LTD,56
562,RU000A0HL5M1,BELUGA GROUP PJSC,BELUGA,52
1008,IL0004560129,UPSELLON BRANDS HOLDINGS LTD,CHIRON REFINERIES LTD,33
...,...,...,...,...
16577,US74838C1062,BOARDRIDERS INC,QUIKSILVER INC,21
16581,US7472623013,QVC INC,QVC INC 6 3/8 09/13/67 PFD,42
16690,SE0010520155,ICTA AB,ROLLING OPTICS HOLDING AB,31
17055,US7847882005,STRATS-FON-04-2,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",38


## Fix the IB Columns

In [28]:
path = "./Data/withFIGI.csv"

df_figi = pd.read_csv(path, index_col=0)  
df_figi

,Name_197,Exchange_197,Country_197,Currency_197,stockType_197,Exchange_197.1,Country_197.1,Currency_197.1,Code-EOD,Country-EOD,...,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI,Not_Match,Source_Available
0,1ST GROUP LTD,ASX,Australia,AUD,COMMON,ASX,Australia,AUD,1ST,Australia,...,BBG008HNLRF4,BBG008HNLRF4,BBG008HNLRG3,AU,Equity,1ST,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
1,2U INC,NASDAQ,United States of America,USD,COMMON,NASDAQ,United States of America,USD,TWOU,USA,...,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock,NaN,FIGI;Inv;IB;EOD;
2,3DX INDUSTRIES INC,PINK,United States of America,USD,COMMON,PINK,United States of America,USD,DDDX,USA,...,BBG000C129N6,BBG000C129N6,BBG001STRH79,US,Equity,DDDX,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
3,3M CO,US Exchange,Mexico,NaN,NaN,US Exchange,Mexico,NaN,MMM,USA,...,BBG000BP52R2,BBG000BP52R2,BBG001S5T7X2,US,Equity,MMM,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
4,88 ENERGY LTD,LSE,United Kingdom,GBP,COMMON,LSE,United Kingdom,GBP,88E,UK,...,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock,NaN,FIGI;Inv;IB;EOD;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19731,ZYNERBA PHARMACEUTICALS INC,NASDAQ,United States of America,USD,COMMON,NASDAQ,United States of America,USD,ZYNE,USA,...,BBG007BBS8B7,BBG007BBS8B7,BBG007BBS8C6,US,Equity,ZYNE,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
19732,ZYNEX INC,NASDAQ,United States of America,USD,COMMON,NASDAQ,United States of America,USD,ZYXI,USA,...,BBG000BJBXZ2,BBG000BJBXZ2,BBG001S7T7V0,US,Equity,ZYXI,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
19733,ZYNGA INC - CL A,NASDAQ,Switzerland,USD,COMMON,NASDAQ,Switzerland,USD,NaN,NaN,...,BBG005YZL376,BBG005YZL2R6,BBG001T24RN2,XS,Equity,ZNGAEUR,Common Stock,Common Stock,NaN,FIGI;IB;
19734,ZYTRONIC PLC,LSE,United Kingdom,GBP,COMMON,LSE,United Kingdom,GBP,ZYT,UK,...,BBG000BZPV76,BBG000BZPTJ8,BBG001SFLP13,LN,Equity,ZYT,Common Stock,Common Stock,NaN,FIGI;IB;EOD;


In [22]:
path = "./Data/LIst of 19.7 Stocks with FIGI if available - By Evelyn.xlsx"

df_stockList = pd.read_excel(path, index_col=0)  
df_stockList

,symbol_x,longName,Exchange,Country,country,name,full_name,isin,currency_x,currency_y,...,Country_Final,Code,figi,compositeFIGI,shareClassFIGI,exchCode,marketSector,securityDescription,securityType,securityType2
0,1ST,1ST GROUP LTD,ASX,Australia,australia,NaN,NaN,NaN,NaN,AUD,...,Australia,1.0,BBG008HNLRF4,BBG008HNLRF4,BBG008HNLRG3,AU,Equity,1ST,Common Stock,Common Stock
1,TWOU,2U INC,NASDAQ,United States of America,united states of america,NaN,NaN,NaN,NaN,USD,...,United States of America,1.0,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
2,DDDX,3DX INDUSTRIES INC,PINK,United States of America,united states of america,NaN,NaN,NaN,NaN,USD,...,United States of America,1.0,BBG000C129N6,BBG000C129N6,BBG001STRH79,US,Equity,DDDX,Common Stock,Common Stock
3,MMM,3M CO,US Exchange,Mexico,mexico,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,88E,88 ENERGY LTD,LSE,United Kingdom,united kingdom,88 Energy Ltd,88 Energy Ltd,AU00000088E2,GBP,GBP,...,United Kingdom,1.0,BBG000C4PMQ3,BBG000C4PMQ3,BBG001SCV1F7,AU,Equity,88E,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19731,ZYNE,ZYNERBA PHARMACEUTICALS INC,NASDAQ,United States of America,united states of america,NaN,NaN,NaN,NaN,USD,...,United States of America,1.0,BBG007BBS8B7,BBG007BBS8B7,BBG007BBS8C6,US,Equity,ZYNE,Common Stock,Common Stock
19732,ZYXI,ZYNEX INC,NASDAQ,United States of America,united states of america,NaN,NaN,NaN,NaN,USD,...,United States of America,1.0,BBG000BJBXZ2,BBG000BJBXZ2,BBG001S7T7V0,US,Equity,ZYXI,Common Stock,Common Stock
19733,ZNGA,ZYNGA INC - CL A,NASDAQ,Switzerland,switzerland,NaN,NaN,NaN,NaN,USD,...,United States of America,1.0,BBG000VD6768,BBG000VD6768,BBG001T24RN2,US,Equity,ZNGA,Common Stock,Common Stock
19734,ZYT,ZYTRONIC PLC,LSE,United Kingdom,united kingdom,NaN,NaN,NaN,NaN,GBP,...,United Kingdom,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_197 = df_stockList[['longName', 'Exchange', 'Country', 'currency_y', 'stockType']]
df_197 = df_197.rename(index=str, columns={'longName':'Name_197'})

df_197

,Name_197,Exchange,Country,currency_y,stockType
0,1ST GROUP LTD,ASX,Australia,AUD,COMMON
1,2U INC,NASDAQ,United States of America,USD,COMMON
2,3DX INDUSTRIES INC,PINK,United States of America,USD,COMMON
3,3M CO,US Exchange,Mexico,NaN,NaN
4,88 ENERGY LTD,LSE,United Kingdom,GBP,COMMON
...,...,...,...,...,...
19731,ZYNERBA PHARMACEUTICALS INC,NASDAQ,United States of America,USD,COMMON
19732,ZYNEX INC,NASDAQ,United States of America,USD,COMMON
19733,ZYNGA INC - CL A,NASDAQ,Switzerland,USD,COMMON
19734,ZYTRONIC PLC,LSE,United Kingdom,GBP,COMMON


In [25]:
df_figi.drop(['Country-IB', 'Exchange-IB', 'Currency-IB'], axis=1, inplace=True)

pd.merge(df_197, df_figi, how='right', on='Name_197')

KeyError: "['Country-IB' 'Exchange-IB' 'Currency-IB'] not found in axis"

In [26]:
df_figi = pd.merge(df_197, df_figi, how='right', on='Name_197')
df_figi = df_figi.rename(index=str, columns={'Exchange':'Exchange_197', 'Country': 'Country_197', 
                                             'currency_y': 'Currency_197', 'stockType': 'stockType_197'})

df_figi

,Name_197,Exchange_197,Country_197,Currency_197,stockType_197,Exchange_197,Country_197,Currency_197,Code-EOD,Country-EOD,...,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI,Not_Match,Source_Available
0,1ST GROUP LTD,ASX,Australia,AUD,COMMON,ASX,Australia,AUD,1ST,Australia,...,BBG008HNLRF4,BBG008HNLRF4,BBG008HNLRG3,AU,Equity,1ST,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
1,2U INC,NASDAQ,United States of America,USD,COMMON,NASDAQ,United States of America,USD,TWOU,USA,...,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock,NaN,FIGI;Inv;IB;EOD;
2,3DX INDUSTRIES INC,PINK,United States of America,USD,COMMON,PINK,United States of America,USD,DDDX,USA,...,BBG000C129N6,BBG000C129N6,BBG001STRH79,US,Equity,DDDX,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
3,3M CO,US Exchange,Mexico,NaN,NaN,US Exchange,Mexico,NaN,MMM,USA,...,BBG000BP52R2,BBG000BP52R2,BBG001S5T7X2,US,Equity,MMM,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
4,88 ENERGY LTD,LSE,United Kingdom,GBP,COMMON,LSE,United Kingdom,GBP,88E,UK,...,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock,NaN,FIGI;Inv;IB;EOD;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19731,ZYNERBA PHARMACEUTICALS INC,NASDAQ,United States of America,USD,COMMON,NASDAQ,United States of America,USD,ZYNE,USA,...,BBG007BBS8B7,BBG007BBS8B7,BBG007BBS8C6,US,Equity,ZYNE,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
19732,ZYNEX INC,NASDAQ,United States of America,USD,COMMON,NASDAQ,United States of America,USD,ZYXI,USA,...,BBG000BJBXZ2,BBG000BJBXZ2,BBG001S7T7V0,US,Equity,ZYXI,Common Stock,Common Stock,NaN,FIGI;IB;EOD;
19733,ZYNGA INC - CL A,NASDAQ,Switzerland,USD,COMMON,NASDAQ,Switzerland,USD,NaN,NaN,...,BBG005YZL376,BBG005YZL2R6,BBG001T24RN2,XS,Equity,ZNGAEUR,Common Stock,Common Stock,NaN,FIGI;IB;
19734,ZYTRONIC PLC,LSE,United Kingdom,GBP,COMMON,LSE,United Kingdom,GBP,ZYT,UK,...,BBG000BZPV76,BBG000BZPTJ8,BBG001SFLP13,LN,Equity,ZYT,Common Stock,Common Stock,NaN,FIGI;IB;EOD;


In [29]:
df_figi.to_csv('./Data/withFIGI.csv')